In [13]:
import os
import json
import os
import base64
import random
from openai import OpenAI
import asyncio
import nest_asyncio
from openai import AsyncOpenAI

nest_asyncio.apply()

In [17]:
# count the number of images in each folder in the photos folder
# path to the photos folder
path = "photos"

# list of folders in the photos folder
folders = os.listdir(path)[2:3] # limit for testing

# loop through the folders
total_images = 0
for folder in folders:
    # ignore if not a directory
    if not os.path.isdir(f"{path}/{folder}"):
        continue
    # get the length of the list of images in the folder
    num_images = len(os.listdir(f"{path}/{folder}"))
    print(f"{folder}: {num_images} images")
    total_images += num_images

print(f"Total images: {total_images}")

coffee shop sydney: 33 images
Total images: 33


In [18]:
# Make sure you have your OPENAI_API_KEY set in your environment variables
# https://platform.openai.com/docs/quickstart?context=python

client = OpenAI()

def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your images - choose at random from folder
def choose_image_pair():
    all_images = []
    for folder in folders:
        # ignore if not a directory
        if not os.path.isdir(f"photos/{folder}"):
            continue
        list_of_images = os.listdir(f"photos/{folder}")
        all_images.extend([f"photos/{folder}/{image}" for image in list_of_images])

    # choose two images at random
    image_path_a = random.choice(all_images)
    image_path_b = random.choice(all_images)

    return image_path_a, image_path_b

image_path_a, image_path_b = choose_image_pair()

print(f"Image A: {image_path_a}")
print(f"Image B: {image_path_b}")
print()

# Getting the base64 string
base64_image_a = encode_image(image_path_a)
base64_image_b = encode_image(image_path_b)

# Get back the labels with inductive coding
def inductive_coding(base64_image_a, base64_image_b):
    user_prompt = """Compare these two coffee shops using inductive coding. What features, attributes, or elements of the design are similar or different? Give a JSONL response with a descriptive label name, coffee_shop_A and coffee_shop_B as keys, and one row per label. The value of the column for each coffee shop should be 1 if the label applies to this coffee shop, or zero if it doesn't. If the label applies to both coffee shops, it should be 1  in both columns. Only output the JSONL."""

    response = client.chat.completions.create(
      model="gpt-4-vision-preview",
      messages=[
        {
          "role": "user",
          "content": [
            {"type": "text", "text": user_prompt},
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image_a}",
              },
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image_b}",
              },
            },
          ],
        }
      ],
      max_tokens=500,
    )

    labels = response.choices[0].message.content
    return labels

labels = inductive_coding(base64_image_a, base64_image_b)
print(labels)

Image A: photos/coffee shop sydney/coffeeshopsydney25.jpeg
Image B: photos/coffee shop sydney/coffeeshopsydney31.jpeg

{"label": "Modern_design", "coffee_shop_A": 1, "coffee_shop_B": 1}
{"label": "Wooden_elements", "coffee_shop_A": 1, "coffee_shop_B": 1}
{"label": "Natural_light", "coffee_shop_A": 0, "coffee_shop_B": 1}
{"label": "Spacious_interior", "coffee_shop_A": 1, "coffee_shop_B": 0}
{"label": "Counter_service", "coffee_shop_A": 1, "coffee_shop_B": 1}
{"label": "Seating_area", "coffee_shop_A": 1, "coffee_shop_B": 0}
{"label": "Suspended_lighting", "coffee_shop_A": 0, "coffee_shop_B": 1}
{"label": "Visible_pastry_display", "coffee_shop_A": 1, "coffee_shop_B": 1}
{"label": "Barista_present", "coffee_shop_A": 1, "coffee_shop_B": 1}
{"label": "Customers_queuing", "coffee_shop_A": 1, "coffee_shop_B": 0}


In [19]:
# create a list of dictionary with the filename, search keyword, label, value for each label
def clean_and_save_labels(labels):
    cleaned_labels = []
    for label in labels.split("\n"):
        label_json = json.loads(label)

        if label_json["coffee_shop_A"] == 1:
            cleaned_labels.append(
                {
                    "filename": image_path_a.split("/")[-1],
                    "folder": image_path_a.split("/")[-2],
                    "label": label_json["label"],
                }
            )
        if label_json["coffee_shop_B"] == 1:
            cleaned_labels.append(
                {
                    "filename": image_path_b.split("/")[-1],
                    "folder": image_path_b.split("/")[-2],
                    "label": label_json["label"],
                }
            )

    # if jsonl file exists, append to the file
    if os.path.exists("cleaned_labels.jsonl"):
        with open("cleaned_labels.jsonl", "a") as file:
            for label in cleaned_labels:
                file.write(json.dumps(label) + "\n")
    else:
        with open("cleaned_labels.jsonl", "w") as file:
            for label in cleaned_labels:
                file.write(json.dumps(label) + "\n")
                
    return cleaned_labels

cleaned_labels = clean_and_save_labels(labels)
print(cleaned_labels)

[{'filename': 'coffeeshopsydney25.jpeg', 'folder': 'coffee shop sydney', 'label': 'Modern_design'}, {'filename': 'coffeeshopsydney31.jpeg', 'folder': 'coffee shop sydney', 'label': 'Modern_design'}, {'filename': 'coffeeshopsydney25.jpeg', 'folder': 'coffee shop sydney', 'label': 'Wooden_elements'}, {'filename': 'coffeeshopsydney31.jpeg', 'folder': 'coffee shop sydney', 'label': 'Wooden_elements'}, {'filename': 'coffeeshopsydney31.jpeg', 'folder': 'coffee shop sydney', 'label': 'Natural_light'}, {'filename': 'coffeeshopsydney25.jpeg', 'folder': 'coffee shop sydney', 'label': 'Spacious_interior'}, {'filename': 'coffeeshopsydney25.jpeg', 'folder': 'coffee shop sydney', 'label': 'Counter_service'}, {'filename': 'coffeeshopsydney31.jpeg', 'folder': 'coffee shop sydney', 'label': 'Counter_service'}, {'filename': 'coffeeshopsydney25.jpeg', 'folder': 'coffee shop sydney', 'label': 'Seating_area'}, {'filename': 'coffeeshopsydney31.jpeg', 'folder': 'coffee shop sydney', 'label': 'Suspended_light

In [20]:
async_client = AsyncOpenAI()

async def process_label(image, folder, path, label):
    base64_image = encode_image(f"{path}/{folder}/{image}")
    user_prompt = f"""If the label "{label['label']}" applies to this image, return 1, otherwise return 0. Only output a 1 or 0."""

    response = await async_client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                        },
                    },
                ],
            }
        ],
        max_tokens=5,
    )

    score = response.choices[0].message.content
    if score == "1":
        print(f"Label {label['label']}")
        return {
                "filename": image,
                "folder": folder,
                "label": label["label"],
            }
    else:
        return None

async def process_image(image, folder, path, cleaned_labels):
    print(f"Processing {image}")
    tasks = [process_label(image, folder, path, label) for label in cleaned_labels]
    result = await asyncio.gather(*tasks)
    print()
    return [label for label in result if label]

async def deductive_coding():
    with open("cleaned_labels.jsonl", "r") as file:
        cleaned_labels = [json.loads(line) for line in file]

     # remove duplicate labels
    cleaned_labels = list({label['label']:label for label in cleaned_labels}.values())

    # limit to first 5 labels for testing
    cleaned_labels = cleaned_labels[0:5]

    processed_labels = []
    for folder in folders:
        if not os.path.isdir(f"{path}/{folder}"):
            continue
        for image in os.listdir(f"{path}/{folder}"):
            if not image.endswith(".jpeg"):
                continue
            result = await process_image(image, folder, path, cleaned_labels)
            processed_labels.extend(result)

    if os.path.exists("processed_labels.jsonl"):
        mode = "a"
    else:
        mode = "w"

    with open("processed_labels.jsonl", mode) as file:
        for label in processed_labels:
            file.write(json.dumps(label) + "\n")

    return processed_labels

async def main():
    processed_labels = await deductive_coding()
    print(f"There are {len(processed_labels)} labels. The first 5 labels are: {processed_labels[:5]}")

asyncio.run(main())

Processing coffeeshopsydney26.jpeg
Label Wooden_tables
Label Natural_light
Label Hanging_light_fixtures

Processing coffeeshopsydney30.jpeg
Label Wooden_tables
Label Natural_light

Processing coffeeshopsydney10.jpeg
Label Hanging_light_fixtures
Label Wooden_tables

Processing coffeeshopsydney31.jpeg
Label Natural_light
Label Hanging_light_fixtures

Processing coffeeshopsydney27.jpeg
Label Natural_light

Processing coffeeshopsydney7.jpeg
Label Natural_light
Label Exposed_beams
Label Wooden_tables
Label Open_ceiling_design
Label Hanging_light_fixtures

Processing coffeeshopsydney16.jpeg
Label Wooden_tables
Label Exposed_beams
Label Open_ceiling_design
Label Natural_light
Label Hanging_light_fixtures

Processing coffeeshopsydney0.jpeg
Label Wooden_tables
Label Natural_light
Label Hanging_light_fixtures

Processing coffeeshopsydney20.jpeg
Label Hanging_light_fixtures
Label Open_ceiling_design

Processing coffeeshopsydney36.jpeg
Label Natural_light
Label Hanging_light_fixtures
Label Exposed

In [24]:
# load into a pandas dataframe
import pandas as pd

df = pd.read_json("processed_labels.json")
df.head()


,filename,folder,label
0,coffeeshopnewyork42.jpeg,coffee shop new york,Street View
1,coffeeshopnewyork42.jpeg,coffee shop new york,Patrons Present
2,coffeeshopnewyork2.jpeg,coffee shop new york,Indoor Plants
3,coffeeshopnewyork2.jpeg,coffee shop new york,Visible Menu on Wall
4,coffeeshopnewyork2.jpeg,coffee shop new york,Pendant Lighting


In [25]:
# count for each label without 'Name: count, dtype: float64' at the bottom
label_counts = df["label"].value_counts() / total_images

for label, count in label_counts.items():
    print(f"{label}: {count}")

Patrons Present: 0.41025641025641024
Visible Menu on Wall: 0.1794871794871795
Pendant Lighting: 0.1794871794871795
Indoor Plants: 0.14102564102564102
Counter Service: 0.14102564102564102
Street View: 0.11538461538461539
Exposed Brick Interior: 0.10256410256410256
